# Preppin Data 2023 W09

https://preppindata.blogspot.com/2023/03/2023-week-9-customer-bank-statements.html

#### Load data

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
tpath = pd.read_csv('Transaction Path.csv')

In [4]:
tdetail = pd.read_csv('Transaction Detail.csv')

In [5]:
accinfo = pd.read_csv('Account Information.csv')

#### Check headers

In [6]:
tpath.head()

,Transaction ID,Account_To,Account_From
0,1957155,27356852,76206810
1,2147025,44242297,24826358
2,3065073,10295384,52104303
3,6622100,45519330,69315008
4,14877473,28680375,44586370


In [7]:
tdetail.head()

,Transaction ID,Transaction Date,Value,Cancelled?
0,1957155,2023-02-01,128.78,N
1,28234510,2023-02-01,163.82,N
2,33688648,2023-02-01,54.71,N
3,41670299,2023-02-01,88.10,N
4,42825784,2023-02-01,217.22,Y


In [8]:
accinfo.head()

,Account Number,Account Type,Account Holder ID,Balance Date,Balance
0,10005367,Platinum,70390615,2023-01-31,728.25
1,10011977,Basic,20123998,2023-01-31,676.54
2,10024680,Platinum,54374080,2023-01-31,567.46
3,10031238,Basic,97027297,2023-01-31,576.52
4,10034341,Joint,"89920386, 97325900",2023-01-31,390.39


#### Change tpath headers

In [9]:
tpath = tpath.rename(columns={'Account_To':'Account To', 'Account_From':'Account From'})

In [10]:
tpath.head()

,Transaction ID,Account To,Account From
0,1957155,27356852,76206810
1,2147025,44242297,24826358
2,3065073,10295384,52104303
3,6622100,45519330,69315008
4,14877473,28680375,44586370


#### Remove cancelled transactions

In [11]:
tdetail.groupby('Cancelled?').count()

,Transaction ID,Transaction Date,Value
Cancelled?,,,
N,8689,8689,8689
Y,92,92,92


In [12]:
tdetail = tdetail.drop(tdetail[tdetail['Cancelled?']=='Y'].index)

In [13]:
tdetail.groupby('Cancelled?').count()

,Transaction ID,Transaction Date,Value
Cancelled?,,,
N,8689,8689,8689


#### merge transactions details

In [14]:
txn = tpath.merge(tdetail, on='Transaction ID', how='left')

#### Create transaction flow tables

In [15]:
# negative number for money leaving account
txn['Account From Amount'] = txn['Value']*-1

In [16]:
txn.head()

,Transaction ID,Account To,Account From,Transaction Date,Value,Cancelled?,Account From Amount
0,1957155,27356852,76206810,2023-02-01,128.78,N,-128.78
1,2147025,44242297,24826358,2023-02-09,170.19,N,-170.19
2,3065073,10295384,52104303,2023-02-06,87.57,N,-87.57
3,6622100,45519330,69315008,2023-02-07,85.76,N,-85.76
4,14877473,28680375,44586370,2023-02-02,84.65,N,-84.65


#### Table for money going into account

In [17]:
txn_from = txn[['Account From','Transaction Date','Account From Amount']]

In [18]:
txn_from = txn_from.rename(columns={'Account From':'Account Number', 'Transaction Date':'Balance Date', \
                         'Account From Amount':'Transaction Value'})

#### Table for money going out of account

In [19]:
txn_to = txn[['Account To','Transaction Date','Value']]

In [20]:
txn_to = txn_to.rename(columns={'Account To':'Account Number', 'Transaction Date':'Balance Date', \
                         'Value':'Transaction Value'})

#### Statement table for starting balances

In [21]:
stmt = accinfo[['Account Number','Balance Date','Balance']]

In [22]:
stmt['Transaction Value'] = np.nan

C:\Users\amar\AppData\Local\Temp\ipykernel_324\2974513272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stmt['Transaction Value'] = np.nan


#### Merge all together

In [23]:
dfs = [stmt, txn_from, txn_to]

In [24]:
accs = pd.concat(dfs)

In [25]:
accs

,Account Number,Balance Date,Balance,Transaction Value
0,10005367,2023-01-31,728.25,NaN
1,10011977,2023-01-31,676.54,NaN
2,10024680,2023-01-31,567.46,NaN
3,10031238,2023-01-31,576.52,NaN
4,10034341,2023-01-31,390.39,NaN
...,...,...,...,...
8776,17925406,2023-02-13,NaN,112.38
8777,37678813,2023-02-13,NaN,78.14
8778,54458410,2023-02-09,NaN,138.80
8779,57426365,2023-02-01,NaN,123.23


#### Sort based on date and account, then perform running sum

In [26]:
accs = accs.sort_values(by=['Balance Date','Account Number','Transaction Value'],\
                       ascending=[True, True, False])

In [27]:
accs

,Account Number,Balance Date,Balance,Transaction Value
0,10005367,2023-01-31,728.25,NaN
1,10011977,2023-01-31,676.54,NaN
2,10024680,2023-01-31,567.46,NaN
3,10031238,2023-01-31,576.52,NaN
4,10034341,2023-01-31,390.39,NaN
...,...,...,...,...
7951,97913385,NaN,NaN,NaN
1754,97983552,NaN,NaN,NaN
1521,98611122,NaN,NaN,NaN
8780,98886699,NaN,NaN,NaN


In [28]:
accs['total'] = (accs['Balance'].fillna(0) + accs['Transaction Value'].fillna(0)).groupby(accs['Account Number']).cumsum(skipna=True)

In [29]:
accs

,Account Number,Balance Date,Balance,Transaction Value,total
0,10005367,2023-01-31,728.25,NaN,728.25
1,10011977,2023-01-31,676.54,NaN,676.54
2,10024680,2023-01-31,567.46,NaN,567.46
3,10031238,2023-01-31,576.52,NaN,576.52
4,10034341,2023-01-31,390.39,NaN,390.39
...,...,...,...,...,...
7951,97913385,NaN,NaN,NaN,581.14
1754,97983552,NaN,NaN,NaN,-1691.41
1521,98611122,NaN,NaN,NaN,1302.30
8780,98886699,NaN,NaN,NaN,-422.56


#### Check value against sample output to see if it's correct

In [30]:
accs[accs['Account Number']==27356852]

,Account Number,Balance Date,Balance,Transaction Value,total
563,27356852,2023-01-31,414.82,NaN,414.82
7174,27356852,2023-02-01,NaN,160.76,575.58
0,27356852,2023-02-01,NaN,128.78,704.36
5070,27356852,2023-02-03,NaN,-99.42,604.94
3512,27356852,2023-02-04,NaN,381.60,986.54
3340,27356852,2023-02-04,NaN,-57.96,928.58
6417,27356852,2023-02-05,NaN,-116.94,811.64
5230,27356852,2023-02-10,NaN,1271.50,2083.14
6920,27356852,2023-02-12,NaN,-41.20,2041.94
5342,27356852,2023-02-12,NaN,-118.80,1923.14


#### Replace Balance

In [31]:
accs['Balance'] = accs['total']

#### Drop Total

In [32]:
accs = accs.drop('total', axis=1)

#### Output

In [33]:
accs.to_csv('2023W09_output.csv')